In [ ]:
import numpy as np
import gillespy2
from gillespy2 import Model, Species, Parameter, Reaction, Event, \
                      EventTrigger, EventAssignment, RateRule, \
                      AssignmentRule, FunctionDefinition
# from gillespy2 import SSACSolver
# from gillespy2 import VariableSSACSolver
# from gillespy2 import TauLeapingSolver
# from gillespy2 import TauHybridSolver
# from gillespy2 import ODESolver

# devils_v1.0

In [ ]:
class Devilsv10(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="devils_v1.0")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="r", expression="1"))
        self.add_parameter(Parameter(name="K", expression="217"))
        self.add_parameter(Parameter(name="ds", expression="0.22"))
        self.add_parameter(Parameter(name="dI", expression="1.28"))
        self.add_parameter(Parameter(name="L", expression="0.75"))
        self.add_parameter(Parameter(name="sigma", expression="0.5"))
        self.add_parameter(Parameter(name="p", expression="0.6"))
        self.add_parameter(Parameter(name="k0", expression="4"))
        self.add_parameter(Parameter(name="vf", expression="1"))
        self.add_parameter(Parameter(name="vt", expression="0"))

        # Variables
        self.add_species(Species(name="S", initial_value=157, mode="discrete"))
        self.add_species(Species(name="E", initial_value=10, mode="discrete"))
        self.add_species(Species(name="I", initial_value=20, mode="discrete"))
        self.add_species(Species(name="W", initial_value=0, mode="discrete"))
        self.add_species(Species(name="R", initial_value=0, mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="birth", reactants={}, products={'S': 1}, propensity_function="r*(S+E+I+W+R)*(1-(S+E+I+W+R)/K)"))
        self.add_reaction(Reaction(name="death_S", reactants={'S': 1}, products={}, rate=self.listOfParameters["ds"]))
        self.add_reaction(Reaction(name="transmission", reactants={'S': 1}, products={'E': 1}, propensity_function="k0*S*I/(S+E+I+W+R)"))
        self.add_reaction(Reaction(name="death_E", reactants={'E': 1}, products={}, propensity_function="ds*E"))
        self.add_reaction(Reaction(name="latency", reactants={'E': 1}, products={'I': 1}, propensity_function="E/L"))
        self.add_reaction(Reaction(name="vaccine_failure", reactants={'W': 1}, products={'E': 1}, propensity_function="k0*sigma*W*I/(S+E+I+W+R)"))
        self.add_reaction(Reaction(name="death_I", reactants={'I': 1}, products={}, propensity_function="dI*I"))
        self.add_reaction(Reaction(name="death_W", reactants={'W': 1}, products={}, propensity_function="ds*W"))
        self.add_reaction(Reaction(name="death_R", reactants={'R': 1}, products={}, propensity_function="ds*R"))

        # Event Triggers
        vaccination_trig = EventTrigger(expression="t >= vt+vf", initial_value=False, persistent=False)

        # Event Assignments
        vaccination_assign_1 = EventAssignment(variable="vt", expression="vt+1")
        vaccination_assign_2 = EventAssignment(variable="W", expression="W+(S*p)")
        vaccination_assign_3 = EventAssignment(variable="S", expression="S*(1-p)")

        # Events
        self.add_event(Event(name="vaccination", trigger=vaccination_trig, assignments=[vaccination_assign_1, vaccination_assign_2, vaccination_assign_3], delay=None, priority="0", use_values_from_trigger_time=True))

        # Timespan
        self.timespan(np.arange(0, 20, 0.05))

In [ ]:
model = Devilsv10()

# Simulation Parameters

In [ ]:
def configure_simulation():
    kwargs = {
        # "solver":TauHybridSolver,
        "number_of_trajectories":100,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

In [ ]:
kwargs = configure_simulation()
results = model.run(**kwargs)

# Visualization

In [ ]:
results.plotplotly()